In [2]:
#1)

import numpy as np

G = np.array([
    [0, 1/2, 0, 0, 1/2, 0, 0, 0, 0, 0, 0, 0],       # A
    [1/3, 0, 1/3, 1/3, 0, 0, 0, 0, 0, 0, 0, 0],     # B
    [0, 1/2, 0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0],       # C
    [1/3, 0, 0, 0, 0, 1/3, 0, 0, 1/3, 0, 0, 0],     # D
    [0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 0, 0, 0],       # E
    [0, 0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 0, 0],       # F
    [0, 0, 1/4, 1/4, 0, 0, 0, 0, 1/4, 0, 0, 1/4],   # G
    [0, 0, 0, 0, 1/5, 1/5, 0, 0, 1/5, 1/5, 1/5, 0], # H
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],           # I
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],           # J
    [0, 0, 0, 0, 0, 0, 0, 0, 1/3, 1/3, 0, 1/3],     # K
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]            # L
])

# Check row sums
row_sums = G.sum(axis=1)
row_sums

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.])

In [3]:
#2: Eigenvalue decomposition
eigvals, eigvecs = np.linalg.eig(G.T)

#3: Find the eigenvector corresponding to the eigenvalue 1
eig_index = np.argmin(np.abs(eigvals - 1))
pi = eigvecs[:, eig_index].real

# Normalize the eigenvector
pi = pi / np.sum(pi)
pi

array([0.06974077, 0.06964627, 0.06823918, 0.13760448, 0.06918413,
       0.05335517, 0.17752289, 0.03492114, 0.12746202, 0.00942325,
       0.00705067, 0.17585004])

In [4]:
#4: Fix dangling nodes
dangling_nodes = np.where(row_sums == 0)[0]
for node in dangling_nodes:
    G[node, :] = 1 / G.shape[1]

# Recompute row sums to ensure they sum to 1
G = G / G.sum(axis=1, keepdims=True)

# Eigenvalue decomposition again
eigvals, eigvecs = np.linalg.eig(G.T)

# Find the eigenvector corresponding to the eigenvalue 1
eig_index = np.argmin(np.abs(eigvals - 1))
pi = eigvecs[:, eig_index].real

#6: Normalize the eigenvector
pi = pi / np.sum(pi)
pi

array([0.0698299 , 0.0698299 , 0.06803939, 0.13697404, 0.0698299 ,
       0.05371531, 0.17547001, 0.03581021, 0.12712623, 0.01074306,
       0.0080573 , 0.17457475])

In [6]:
#9: Not strongly connected
alpha = 0.85
n = G.shape[1]
E = np.ones((n, n)) / n
G_tilde = alpha * G + (1 - alpha) * E

#10: Eigenvalue decomposition of G_tilde
eigvals, eigvecs = np.linalg.eig(G_tilde.T)

# Find the eigenvector corresponding to the eigenvalue 1
eig_index = np.argmin(np.abs(eigvals - 1))
pi_tilde = eigvecs[:, eig_index].real

# Normalize the eigenvector
pi_tilde = pi_tilde / np.sum(pi_tilde)
pi_tilde

array([0.0715815 , 0.07325075, 0.06650942, 0.12799417, 0.07816056,
       0.05894978, 0.14679029, 0.04778035, 0.12162373, 0.02911212,
       0.02268477, 0.15556256])

In [8]:
#5)

# Term-document matrix (T) and query vector (q)
keywords = [
    ["Apples", "Bananas", "Broccoli", "Cabbage", "Kumquats", "Strawberries"],                # Page A
    ["Oranges", "Plums", "Coconuts", "Kumquats", "Blueberries", "Cherries", "Strawberries"], # Page B
    ["Lettuce", "Spinach", "Bananas", "Blackberries", "Peas", "Strawberries"],               # Page C
    ["Oranges", "Onions", "Celery", "Kumquats", "Corn", "Radishes"],                         # Page D
    ["Pineapples", "Plums", "Corn", "Cherries", "Broccoli", "Peas", "Strawberries"],         # Page E
    ["Lettuce", "Onions", "Coconuts", "Spinach", "Peas", "Strawberries"],                    # Page F
    ["Apples", "Onions", "Broccoli", "Corn", "Cabbage", "Peas"],                             # Page G
    ["Plums", "Blueberries", "Raspberries", "Blackberries", "Strawberries"],                 # Page H
    ["Apples", "Cucumbers", "Carrots", "Spinach", "Corn", "Black Beans", "Cabbage"],         # Page I
    ["Mushrooms", "Carrots", "Lettuce", "Radishes", "Peppers", "Broccoli", "Spinach"],       # Page J
    ["Carrots", "Lettuce", "Celery", "Onions", "Cabbage", "Peas"],                           # Page K
    ["Broccoli", "Cabbage", "Carrots", "Spinach", "Corn", "Peas"]                            # Page L
]

# Creating the term-document matrix T
all_keywords = list(set(sum(keywords, [])))
T = np.zeros((len(all_keywords), len(keywords)))

for j, page_keywords in enumerate(keywords):
    for keyword in page_keywords:
        i = all_keywords.index(keyword)
        T[i, j] = 1

# Example user query
query_keywords = ["Apples", "Oranges"]
q = np.zeros(len(all_keywords))
for keyword in query_keywords:
    if keyword in all_keywords:
        i = all_keywords.index(keyword)
        q[i] = 1

# Compute d^T = q^T T
d = np.dot(q, T)

# Responsive webpages
responsive_webpages = np.where(d > 0)[0]

# Rank the responsive webpages using pi_tilde
pi_tilde = pi_tilde
responsive_ranking = np.argsort(-pi_tilde[responsive_webpages])
responsive_webpages_ranked = responsive_webpages[responsive_ranking]

responsive_webpages_ranked

array([6, 3, 8, 1, 0], dtype=int64)

In [2]:
# 1. Define the raw Google Matrix, G

import numpy as np

G = np.array([
    [0, 1/2, 0, 0, 1/2, 0, 0, 0, 0, 0, 0, 0],       # A
    [1/3, 0, 1/3, 1/3, 0, 0, 0, 0, 0, 0, 0, 0],     # B
    [0, 1/2, 0, 1/2, 0, 0, 0, 0, 0, 0, 0, 0],       # C
    [1/3, 0, 0, 0, 0, 1/3, 0, 0, 1/3, 0, 0, 0],     # D
    [0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 0, 0, 0],       # E
    [0, 0, 0, 0, 1/2, 0, 0, 0, 1/2, 0, 0, 0],       # F
    [0, 0, 1/4, 1/4, 0, 0, 0, 0, 1/4, 0, 0, 1/4],   # G
    [0, 0, 0, 0, 1/5, 1/5, 0, 0, 1/5, 1/5, 1/5, 0], # H
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],           # I
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],           # J
    [0, 0, 0, 0, 0, 0, 0, 0, 1/3, 1/3, 0, 1/3],     # K
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]            # L
])


Google Matrix (G):
[[0.         0.5        0.         0.         0.5        0.
  0.         0.         0.         0.         0.         0.        ]
 [0.33333333 0.         0.33333333 0.33333333 0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.5        0.         0.5        0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.33333333 0.         0.         0.         0.         0.33333333
  0.         0.         0.33333333 0.         0.         0.        ]
 [0.         0.         0.         0.5        0.         0.
  0.         0.5        0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.5        0.
  0.         0.         0.5        0.         0.         0.        ]
 [0.         0.         0.25       0.25       0.         0.
  0.         0.         0.25       0.         0.         0.25      ]
 [0.         0.         0.         0.         0.2        0.2
  0.     

In [ ]:

# 2. Fix dangling nodes (rows with all zeros), and normalize
n = G.shape[0]
for i in range(n):
    if np.sum(G[i, :]) == 0:
        G[i, :] = 1 / n  # Distribute to all nodes

# Normalize G to ensure all rows sum to 1
G = G / G.sum(axis=1, keepdims=True)



In [ ]:
# 3. Power method to find the principal eigenvector (PageRank vector)
def power_method(G, eps=1e-6, max_iter=100):
    n = G.shape[0]
    pi = np.ones(n) / n
    for i in range(max_iter):
        pi_new = G.T @ pi
        if np.linalg.norm(pi_new - pi, 1) < eps:
            break
        pi = pi_new
    return pi / np.sum(pi)

pi = power_method(G)


In [ ]:

# 4. Construct the modified Google matrix with teleportation
alpha = 0.85
E = np.ones((n, n)) / n
G_tilde = alpha * G + (1 - alpha) * E


In [ ]:

# 5. Power method on G_tilde
pi_tilde = power_method(G_tilde)


In [ ]:

# 6. Normalization
pi = pi / np.sum(pi)
pi_tilde = pi_tilde / np.sum(pi_tilde)


In [ ]:

# 7. Keyword data
keywords = [
    ["Apples", "Bananas", "Broccoli", "Cabbage", "Kumquats", "Strawberries"],                # Page A,1
    ["Oranges", "Plums", "Coconuts", "Kumquats", "Blueberries", "Cherries", "Strawberries"], # Page B,2
    ["Lettuce", "Spinach", "Bananas", "Blackberries", "Peas", "Strawberries"],               # Page C,3
    ["Oranges", "Onions", "Celery", "Kumquats", "Corn", "Radishes"],                         # Page D,4
    ["Pineapples", "Plums", "Corn", "Cherries", "Broccoli", "Peas", "Strawberries"],         # Page E,5
    ["Lettuce", "Onions", "Coconuts", "Spinach", "Peas", "Strawberries"],                    # Page F,6
    ["Apples", "Onions", "Broccoli", "Corn", "Cabbage", "Peas"],                             # Page G,7
    ["Plums", "Blueberries", "Raspberries", "Blackberries", "Strawberries"],                 # Page H,8
    ["Apples", "Cucumbers", "Carrots", "Spinach", "Corn", "Black Beans", "Cabbage"],         # Page I,9
    ["Mushrooms", "Carrots", "Lettuce", "Radishes", "Peppers", "Broccoli", "Spinach"],       # Page J,10
    ["Carrots", "Lettuce", "Celery", "Onions", "Cabbage", "Peas"],                           # Page K,11
    ["Broccoli", "Cabbage", "Carrots", "Spinach", "Corn", "Peas"]                            # Page L,12
]

# User query
query = ["Apple", "Oranges"]


In [ ]:

# 8. Construct Term-Document Matrix T
terms = list(set([term for sublist in keywords for term in sublist]))  # Unique terms
T = np.zeros((len(terms), n))
for j in range(n):
    for term in keywords[j]:
        i = terms.index(term)
        T[i, j] = 1


In [ ]:

# 9. Create query vector q
q = np.zeros(len(terms))
for term in query:
    if term in terms:
        q[terms.index(term)] = 1


In [ ]:

# 10. Compute dᵀ = qᵀ T
d = q.T @ T


In [ ]:

# 11. Rank webpages by importance and responsiveness
responsive_pages = np.argsort(-d)
importance_ranking = np.argsort(-pi_tilde)


In [ ]:

# 12. Final ranking of responsive webpages based on importance
ranked_responsive_pages = [page for page in importance_ranking if d[page] > 0]

# Output results
print("Google Matrix (G):")
print(G)
print("\nPageRank Vector (pi):")
print(pi)
print("\nModified Google Matrix (G_tilde):")
print(G_tilde)
print("\nModified PageRank Vector (pi_tilde):")
print(pi_tilde)
print("\nQuery Vector (q):")
print(q)
print("\nTerm-Document Matrix (T):")
print(T)
print("\nResponsiveness Vector (d):")
print(d)
print("\nFinal Ranking of Responsive Webpages:")
print(ranked_responsive_pages)